In [266]:
from numpy import *
from scipy import *
from scipy.sparse import *
from builtins import *

In [247]:
tracks_with_tags=load("tracks_with_tags.npy")
tracks_with_tags[0]

array([2972914,   54087,    1757,    1718,  116712,  189631], dtype=int32)

In [248]:
sample_track_tags=tracks_with_tags[0][1:]
sample_track_tags

array([ 54087,   1757,   1718, 116712, 189631], dtype=int32)

In [358]:
playlists_with_tags=load_npz("playlists_with_tags.npz")
sample_playlist=playlists_with_tags[0]
sample_playlist

<1x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 66 stored elements in Compressed Sparse Column format>

In [253]:
sample_playlist.data

array([  1.,   1.,   2.,   1.,   1.,   1.,  22.,   1.,  20.,   5.,   1.,
         2.,   1.,  20.,   5.,  52.,   1.,   1.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   6.,   1.,   1.,   1.,   1.,  53.,   6.,   1.,
        12.,   1.,   1.,   1.,   1.,   2.,   1.,   1.,   2.,   1.,  10.,
         1.,   1.,   1.,   1.,   1.,  35.,   2.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   1.,   1.,   1.,   2.,   1.,   1.,   1.,   2.])

In [251]:
sample_playlist_tags=sample_playlist.nonzero()[1] # second column (first is of zeros)
sample_playlist_tags

array([  1264,   1280,   1319,   2942,   3838,   3982,   4425,   6120,
        11056,  11242,  17739,  24358,  25437,  35060,  35129,  35233,
        35259,  41236,  50604,  50764,  59903,  65509,  70618,  73151,
        79716,  81223,  83064,  86574,  99566, 109806, 115355, 115684,
       115695, 115752, 118582, 138704, 144192, 144388, 147126, 149136,
       170061, 175803, 186445, 189631, 191686, 194264, 195027, 196918,
       197909, 205245, 216938, 226381, 228514, 230451, 235198, 242525,
       244317, 246452, 248830, 258302, 262263, 267005, 274189, 275454,
       275470, 275509], dtype=int32)

In [258]:
# non perdere gli indexes
sample_mean=mean(unique(sample_playlist.data))
sample_mean

19.818181818181817

In [259]:
sample_filtered_playlist_tags=sample_playlist[sample_playlist>sample_mean]
sample_filtered_playlist_tags

matrix([[ 22.,  20.,  20.,  52.,  53.,  35.]])

In [348]:
p=playlists_with_tags[0]
print(p.data, p.shape)
firstMean=mean((p.data))
print(firstMean)
secondMean=mean(p[p>firstMean])
print(p[p>firstMean])
print(secondMean)
indexes=where(p.todense()>secondMean)[1]
indexes
p[p>secondMean]
#tentativo con 2 medie che non mi piace

[  1.   1.   2.   1.   1.   1.  22.   1.  20.   5.   1.   2.   1.  20.   5.
  52.   1.   1.   1.   1.   1.   1.   1.   1.   1.   6.   1.   1.   1.   1.
  53.   6.   1.  12.   1.   1.   1.   1.   2.   1.   1.   2.   1.  10.   1.
   1.   1.   1.   1.  35.   2.   1.   1.   1.   1.   1.   1.   1.   1.   1.
   1.   2.   1.   1.   1.   2.] (1, 276615)
4.65151515152
[[ 22.  20.   5.  20.   5.  52.   6.  53.   6.  12.  10.  35.]]
20.5


matrix([[ 22.,  52.,  53.,  35.]])

In [356]:
p=playlists_with_tags[0]
print(p.data)
myMedian=median(unique(p.data))
print(myMedian)
print(p[p>myMedian])
#mediana che mi piace

[  1.   1.   2.   1.   1.   1.  22.   1.  20.   5.   1.   2.   1.  20.   5.
  52.   1.   1.   1.   1.   1.   1.   1.   1.   1.   6.   1.   1.   1.   1.
  53.   6.   1.  12.   1.   1.   1.   1.   2.   1.   1.   2.   1.  10.   1.
   1.   1.   1.   1.  35.   2.   1.   1.   1.   1.   1.   1.   1.   1.   1.
   1.   2.   1.   1.   1.   2.]
12.0
[[ 22.  20.  20.  52.  53.  35.]]


In [180]:
intersect1d(sample_track,sample_tags_median)

array([], dtype=int64)

In [102]:
intersect1d(sample_track_tags,sample_playlist_tags)

array([189631], dtype=int32)

In [103]:
tracks_ordered_indexes=load("indexsortedDESCENDENT.npy")
tracks_ordered_indexes

array([43529, 94738, 57501, ..., 17820, 79618, 43000])

In [330]:
sample_track_tags_2=tracks_with_tags[tracks_ordered_indexes[0]][1:]
sample_track_tags_2

array([255208,  43212,  70618,  64978,  70625], dtype=int32)

In [329]:
commond_tags=intersect1d(sample_track_tags_2,sample_playlist_tags)
print(commond_tags, commond_tags.size)

[70618] 1


# qui sopra esempio di come trovare tag in comune tra canzone e playlist, qui sotto provo a vedere quanto ci mettono due for annidati

In [364]:
# ciclo che scorre le tracce in ordine di popolarità e usa la media
p_ok=0
for p in playlists_with_tags:
    tagsMean=mean(p.data)
    pTags=where(p.todense()>tagsMean)[1]
    tagSize=pTags.size
    found=0
    epsilon=0.79
    if (tagSize<5):
        threshold=tagSize*epsilon
    else:
        threshold=5*epsilon
    for t in tracks_ordered_indexes:
        if (found==5):
            p_ok+=1
            break
        tTags=tracks_with_tags[t][1:]
        if ((intersect1d(pTags,tTags)).size>=threshold):
            found+=1
    if (p_ok%100==0):
        print(p_ok)
        

/home/oppy/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/oppy/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


KeyboardInterrupt: 

In [368]:
def removeStupidValues(p):
    tagsMedian=median(unique(p.data))
    pTags=where(p.todense()>tagsMedian)[1]
    tagSize=pTags.size
    found=0
    epsilon=0.79
    if (tagSize<5):
        threshold=tagSize*epsilon
    else:
        threshold=5*epsilon
    for t in tracks_ordered_indexes:
        if (found==5):
            p_ok+=1
            break
        tTags=tracks_with_tags[t][1:]
        if ((intersect1d(pTags,tTags)).size>=threshold):
            found+=1
    if (p_ok%100==0):
        print(p_ok)

p_ok=0
apply_along_axis(removeStupidValues,0,playlists_with_tags)

AxisError: axis 0 is out of bounds for array of dimension 0

In [363]:
# ciclo che scorre le tracce in ordine di popolarità e usa la mediana
p_ok=0
for p in playlists_with_tags:
    tagsMedian=median(unique(p.data))
    pTags=where(p.todense()>tagsMedian)[1]
    tagSize=pTags.size
    found=0
    epsilon=0.79
    if (tagSize<5):
        threshold=tagSize*epsilon
    else:
        threshold=5*epsilon
    for t in tracks_ordered_indexes:
        if (found==5):
            p_ok+=1
            break
        tTags=tracks_with_tags[t][1:]
        if ((intersect1d(pTags,tTags)).size>=threshold):
            found+=1
    if (p_ok%100==0):
        print(p_ok)
        

/home/oppy/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/oppy/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """


100


KeyboardInterrupt: 